In [26]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

In [27]:
query="""
with parent as (select
			sku.sku_id as sku_id,
			po.name as product_option,
			pov.attribute_value as product_option_value_parent
		from
			postgres_broadleaf_federate."broadleaf.blc_sku" sku
			inner join postgres_broadleaf_federate."broadleaf.blc_sku_option_value_xref" x on sku.sku_id = x.sku_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option_value" pov on x.product_option_value_id = pov.product_option_value_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option" po on pov.product_option_id = po.product_option_id
			
		where
			sku.addl_product_id is not null
			and po.name ilike '%-Parent'
			and pov.attribute_value not ilike '%No Aplicable%'
			--and x.sndbx_archived_flag = 'N' and x.sndbx_deleted_flag = 'N' 
			and x.archived = 'N'
			and x.sndbx_id is null
			),
	child as ( select
			sku.sku_id as sku_id,
			po.name as product_option,
			pov.attribute_value as product_option_value_child
		from
			postgres_broadleaf_federate."broadleaf.blc_sku" sku
			inner join postgres_broadleaf_federate."broadleaf.blc_sku_option_value_xref" x on sku.sku_id = x.sku_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option_value" pov on x.product_option_value_id = pov.product_option_value_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option" po on pov.product_option_id = po.product_option_id
		where
			sku.addl_product_id is not null
			and po.name not ilike '%-Parent'
			and pov.attribute_value not ilike '%No Aplicable%'
			--and x.sndbx_archived_flag = 'N' and x.sndbx_deleted_flag = 'N' 
			and x.archived = 'N'
			and x.sndbx_id is null
	    )
	    
select distinct 
split_part(bs.upc, '-',1) as region
,bs.sku_id
,bs.date_created fecha_creacion
,split_part(bs.upc, ':',4) as IdProd
,bs.addl_product_id as ID_Tarjeta
,bs.upc
,bp.default_category_id
,bcat2.name as categoria
,CASE
	WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
	WHEN bcat.name ILIKE 'Res%' THEN 'Res'
	WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
	ELSE bcat.name
END AS subcategoria
,fs.owner_id
,fow.name as owner_name
,fbb.name as marca
--bs.name, 
--product_option_value_parent, 
--product_option_value_child, 
,bs2.name as card_name
--,concat(bs2.name,' ', product_option_value_parent,' ' , product_option_value_child) as name
,upper(fp.unit) as unidad
,fp.iva
,fs.step_unit
,fs.min_weight_unit
,fs.conversion_type_x as Type_x
,fs.conversion_value_x as Val_x
,fs.conversion_type_y as Type_y
,fs.conversion_value_y as Val_y
,bs.weight as peso
,bs.weight_unit_of_measure as unidad_peso
,        CASE
            WHEN ((bs.active_end_date IS NULL OR bs.active_end_date > CURRENT_TIMESTAMP) AND (bs2.active_end_date IS NULL OR bs2.active_end_date > CURRENT_TIMESTAMP)) THEN 'on'
            ELSE 'off'
        END as status_frida

from postgres_broadleaf_federate."broadleaf.blc_sku" bs
left join postgres_broadleaf_federate."broadleaf.fb_sku" fs on fs.sku_id = bs.sku_id 
LEFT join child on child.sku_id = bs.sku_id
LEFT join parent on child.sku_id = parent.sku_id
LEFT join postgres_broadleaf_federate."broadleaf.blc_product" bp  on bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_product" fp on fp.product_id = bp.product_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_owner" fow on fow.owner_id = fs.owner_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku" bs2 on bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product_attribute bpa on bpa.product_id = bp.product_id and bpa.name = 'Brand' and bpa.archived = 'N' and bpa.sndbx_tier is null
left join postgres_broadleaf_federate."broadleaf.blc_category_xref" bcx on bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and default_reference = 'true' --Relaciones categorias 
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat on bp.default_category_id = bcat.category_id --Nombre subcategoria
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat2 on bcx.category_id = bcat2.category_id --Nombre categoria
left join postgres_broadleaf_federate."broadleaf.fb_brand" fbb on fbb.brand_id = fp.brand_id

where 
bs.archived = 'N'
and bs.sndbx_id is null
and bp.archived = 'N'
and bs.catalog_disc < 0
and ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))"""

frida_products = read_connection_data_warehouse.runQuery(query)

In [28]:
df_margen = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144493,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_margen = pd.merge(df_margen, frida_products,  how='left', left_on=['region','sku'], right_on = ['region','upc'])
df_margen[["gmv_value","cogs_value"]] = df_margen[["gmv_value","cogs_value"]].astype(float)

200


In [29]:
# calculate the profit margin by sku
df_margen2 = df_margen.groupby(by=['region','id_tarjeta']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum() if x['gmv_value'].sum() != 0 else 0).reset_index(name='GM')

# calculate the profit margin by subcategory
df_margen3 = df_margen.groupby(by=['region','subcategoria']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum()).reset_index(name='GM')

/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [30]:
query = """
WITH exploded_data AS (
SELECT DISTINCT
    bs.site_identifier_value AS region_code,
    bo.offer_id,
    MAX(bcs.customer_segment_id) AS customer_segment_id_test,
    --bcs.customer_segment_name,
    MAX(bcs2.customer_segment_id) AS customer_segment_id_control,
    --bcs2.customer_segment_name,
    bo.offer_name AS discount,
    DATE(bo.start_date) AS start_date,
    DATE(bo.end_date) AS end_date,
    bo.offer_value::float as Value_Discount,
    replace(
        substring(
          boic.order_item_match_rule, 
          position('["' in boic.order_item_match_rule) + 2, 
          position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
    ), '"', '') AS values_string
    
FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs     ON bocsx.customer_segment_id = bcs.customer_segment_id AND bcs.customer_segment_name ILIKE '%piloto%'
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs2    ON REPLACE(bcs.customer_segment_name, 'piloto', 'control') = bcs2.customer_segment_name 

WHERE offer_name ILIKE '%\_NB\_%'
    AND end_date >= DATE_TRUNC('month', date(getdate()) - interval '6 month')
    AND end_date < DATE_TRUNC('month', date(getdate()) - interval '5 month')
    AND bo.offer_discount_type = 'PERCENT_OFF'
    AND bo.offer_type = 'ORDER_ITEM'
    AND bo.sndbx_id is NULL
    AND (bo.archived is NULL OR bo.archived='N')
    AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
    AND bo.automatically_added = 'true'
    --AND bs.site_identifier_value = 'MDE'
GROUP BY 1,2,5,6,7,8,9
), 
 
number_series AS (
  SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
  UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20
  UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL SELECT 30
  UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL SELECT 40
  UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL SELECT 50
  UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL SELECT 60
  UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL SELECT 70
  UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL SELECT 80
  UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL SELECT 90
  UNION ALL SELECT 91 UNION ALL SELECT 92 UNION ALL SELECT 93 UNION ALL SELECT 94 UNION ALL SELECT 95 UNION ALL SELECT 96 UNION ALL SELECT 97 UNION ALL SELECT 98 UNION ALL SELECT 99 UNION ALL SELECT 100
)

SELECT DISTINCT
    ed.region_code,
    ed.offer_id,
    ed.customer_segment_id_test,
    ed.customer_segment_id_control,
    ed.discount,
    ed.start_date,
    ed.end_date,
    ed.value_discount,
    bs.addl_product_id AS card_id,
    bs2.name,
    bcat2.name AS cat,
    bcat.name as subcat
    
FROM exploded_data                                                                  ed
JOIN number_series                                                                  ns      ON ns.n <= regexp_count(ed.values_string, ',') + 1 AND trim(split_part(ed.values_string, ',', ns.n)) <> ''
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = trim(split_part(ed.values_string, ',', ns.n))::int
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id

WHERE
    -- FIX SUPER DESCUENTOS
    bcat2.category_id NOT IN ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
"""#.format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

df_offers = read_connection_data_warehouse.runQuery(query)
df_offers['start_date'] = pd.to_datetime(df_offers['start_date'])
df_offers['end_date'] = pd.to_datetime(df_offers['end_date'])

In [31]:
df_offers.head()

,region_code,offer_id,customer_segment_id_test,customer_segment_id_control,discount,start_date,end_date,value_discount,card_id,name,cat,subcat
0,SPO,-14468037,91695,91730,20221101_SPO_grillchurrascariaespetinho_piloto...,2022-11-01,2022-11-23,10.0,135711,Maionese Predilecta Sachê 144x7g,Mercearia,Condimentos
1,GDL,-14466001,89966,89994,20221101_GDL_other_piloto_NB_frutas_W44,2022-11-01,2022-11-30,18.0,179628,Piña Miel,Frutas & Verduras,Frutas
2,BAQ,-14466429,90266,90290,20221101_BAQ_comidaespecializada_piloto_NB_pap...,2022-11-01,2022-11-30,10.0,224615,Servilleta Popular x 330 und,Aseo e Higiene,Papel & Toallas
3,SPO,-14468113,91619,91658,20221101_SPO_boteco_piloto_NB_azeitesoleosevin...,2022-11-01,2022-11-23,6.0,293209,Óleo Vitaliv 900ml,Mercearia,"Azeites, Óleos e Vinagres"
4,VCP,-14466856,91018,91042,20221101_VCP_other_piloto_NB_azeitesoleosevina...,2022-11-01,2022-11-30,6.0,330762,Vinagre de Álcool Galo de Barcelos 750ml,Mercearia,"Azeites, Óleos e Vinagres"


In [32]:
df_offers.loc[df_offers.start_date == df_offers.start_date.max()].region_code.value_counts()

BOG    1720
SPO    1599
CMX    1363
BAQ     998
BHZ     170
CWB     150
GDL     125
PBC     119
VCP     104
MDE      88
Name: region_code, dtype: int64

In [33]:
df_offers_grouped = df_offers.groupby(by = ['region_code', 'offer_id', 'customer_segment_id_test',
       'customer_segment_id_control', 'discount', 'start_date', 'end_date',
       'value_discount']).agg({'card_id': lambda x: x.unique().tolist(), 'name': lambda x: x.unique().tolist()}).reset_index()

In [34]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [35]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment" ba          ON ba.order_id = bo.order_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    --AND bo.submit_date >= DATE_TRUNC('year', date(getdate()) - interval '0 year')
    AND bo.submit_date >= DATE_TRUNC('month', date(getdate()) - interval '6 month')
    AND bo.submit_date < DATE_TRUNC('month', date(getdate()) - interval '5 month')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    --AND bo.customer_id IN {clientes}
)

SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    DATE(bo.submit_date) AS submit_date,
    bo.order_id,
    bs.addl_product_id AS padre_sku_id,
    bcat2.name AS cat,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    bs2.name,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'GDL' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'PBC' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'MTY' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'SPO' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'BHZ' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'CWB' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'VCP' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
    ELSE (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776 END AS gmv_usd,    
    CASE 
        WHEN s.site_identifier_value = 'CMX' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'GDL' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'PBC' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'MTY' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'SPO' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
        WHEN s.site_identifier_value = 'BHZ' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
        WHEN s.site_identifier_value = 'CWB' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
        WHEN s.site_identifier_value = 'VCP' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    ELSE (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/3776 END AS discount_usd,
    (boi.quantity*foi.step_unit) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
"""#.format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

df_sells = read_connection_data_warehouse.runQuery(query)
df_sells[["gmv_usd", "cant","discount_usd"]] = df_sells[["gmv_usd", "cant","discount_usd"]].astype(float)
df_sells['submit_date'] = pd.to_datetime(df_sells['submit_date']) 

In [36]:
print(df_sells.submit_date.min(),df_sells.submit_date.max())

2022-11-01 00:00:00 2022-11-30 00:00:00


In [37]:
df_sells.columns

Index(['city', 'customer_id', 'submit_date', 'order_id', 'padre_sku_id', 'cat',
       'subcat', 'name', 'gmv_usd', 'discount_usd', 'cant'],
      dtype='object')

In [38]:
def df_info_filter_offer(df_offers_grouped, i, test, control):
    df_filter_test = df_sells[(df_sells['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_sells['customer_id'].isin(test)) & 
                         (df_sells['submit_date'] >= df_offers_grouped.loc[i,'start_date']) & 
                         (df_sells['submit_date'] < df_offers_grouped.loc[i,'end_date']) & 
                         (df_sells['padre_sku_id'].isin(df_offers_grouped.loc[i,'card_id']))].copy()
    df_filter_control = df_sells[(df_sells['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_sells['customer_id'].isin(control)) & 
                         (df_sells['submit_date'] >= df_offers_grouped.loc[i,'start_date']) & 
                         (df_sells['submit_date'] < df_offers_grouped.loc[i,'end_date']) & 
                         (df_sells['padre_sku_id'].isin(df_offers_grouped.loc[i,'card_id']))].copy()
    
    if len(df_filter_control) == 0 and len(df_filter_test) == 0:
        return "nada"
    elif len(df_filter_control) == 0:
        dic = {}
        dic = df_filter_test.iloc[0].to_dict()
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_control = pd.concat([df_filter_control, new_df], ignore_index=True)
    elif len(df_filter_test) == 0:
        dic = {}
        dic = df_filter_control.iloc[0].to_dict()
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_test = pd.concat([df_filter_test, new_df], ignore_index=True)
        
    df_filter_test['group'] = 'test'
    df_filter_control['group'] = 'control'
    df_filter_test['all_customer'] = len(test)
    df_filter_control['all_customer'] = len(control)
    df_filter = pd.concat([df_filter_test, df_filter_control]).reset_index(drop=True)
    df_filter['offer_id'] = df_offers_grouped.loc[i,'offer_id']
    #df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: 0.1 if len(df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()) == 0 else df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    df_filter['gmv_usd_GM'] = df_filter.gmv_usd * df_filter.GM
    
    return df_filter

In [40]:
df_offers_grouped[['start_date','end_date']].drop_duplicates().sort_values(by=['start_date'])

,start_date,end_date
0,2022-11-01,2022-11-30
769,2022-11-01,2022-11-23


In [41]:
df_offers_grouped = df_offers_grouped.loc[(df_offers_grouped.start_date >= '2022-11-01') & (df_offers_grouped.end_date <= '2022-12-01')].reset_index(drop=True).copy()

In [42]:
global_results_list = [] 
columns = ['city', 'offer_id', 'subcat', 'group', 'customer_count', 'gmv_usd',
       'discount_usd', 'quantity', 'all_customer',
       'convertion_rate', 'AOV', 'dct_avg']
results_df = pd.DataFrame()#data={col: [] for col in columns})

for i in range(len(df_offers_grouped)):
    test = customers_offer(df_offers_grouped.loc[i,'customer_segment_id_test'])
    control = customers_offer(df_offers_grouped.loc[i,'customer_segment_id_control'])
    print(df_offers_grouped.loc[i,'offer_id'])
    
    info_offer = df_info_filter_offer(df_offers_grouped, i, test, control)
    if type(info_offer) == str:
     pass 
    else:
     results_offer = (info_offer.groupby(by=['city', 'offer_id', 'subcat', 'group', 'all_customer'])
                                   .agg(customer_count=('customer_id', 'nunique'),
                                        gmv_usd=('gmv_usd', np.sum),
                                        discount_usd=('discount_usd', np.sum),
                                        quantity=('cant', np.sum),
                                        margen=('GM', np.mean),
                                        gmv_usd_GM=('gmv_usd_GM', np.sum))
                                   .reset_index()
                         )
     
     results_offer['convertion_rate'] = results_offer.customer_count / results_offer.all_customer       
     results_offer['AOV'] = results_offer.gmv_usd / results_offer.customer_count
     results_offer['AOV_net'] = results_offer.gmv_usd_GM / results_offer.customer_count
     results_offer['dct_avg'] = results_offer.discount_usd / results_offer.customer_count
     
     results_df = pd.concat([results_df, results_offer]).reset_index(drop=True)

-14466693
-14466692
-14466691
-14466690
-14466689
-14466688
-14466687
-14466686
-14466685
-14466684
-14466683
-14466682
-14466681
-14466680
-14466679
-14466678
-14466677
-14466676
-14466675
-14466674
-14466673
-14466672
-14466671
-14466670
-14466669
-14466643
-14466642
-14466641
-14466640
-14466639
-14466638
-14466637
-14466636
-14466635
-14466634
-14466633
-14466632
-14466631
-14466630
-14466629
-14466628
-14466627
-14466626
-14466625
-14466624
-14466623
-14466601
-14466600
-14466599
-14466598
-14466597
-14466596
-14466595
-14466594
-14466593
-14466592
-14466591
-14466590
-14466589
-14466588
-14466587
-14466586
-14466585
-14466584
-14466583
-14466582
-14466581
-14466580
-14466579
-14466578
-14466577
-14466576
-14466575
-14466574
-14466573
-14466543
-14466542
-14466541
-14466540
-14466539
-14466538
-14466537
-14466536
-14466535
-14466534
-14466533
-14466532
-14466531
-14466530
-14466529
-14466528
-14466527
-14466526
-14466525
-14466524
-14466523
-14466522
-14466521
-14466520
-14466519


: 

In [ ]:
results_df.head()

,city,offer_id,subcat,group,all_customer,customer_count,gmv_usd,discount_usd,quantity,margen,gmv_usd_GM,convertion_rate,AOV,AOV_net,dct_avg
0,BAQ,-14487713,Mantequillas,control,108,1,5.033633,0.000000,1.0,0.174775,0.879753,0.009259,5.033633,0.879753,0.000000
1,BAQ,-14487713,Mantequillas,test,437,3,75.504502,4.026907,15.0,0.174775,13.196293,0.006865,25.168167,4.398764,1.342302
2,BAQ,-14487677,Aceites & Grasas,control,40,7,92.797669,1.855953,17.0,0.079473,7.374873,0.175000,13.256810,1.053553,0.265136
3,BAQ,-14487677,Aceites & Grasas,test,159,65,1166.403713,53.126456,213.0,0.078452,91.007690,0.408805,17.944673,1.400118,0.817330
4,BAQ,-14487676,Arroz,control,72,7,346.980932,13.742055,26.0,0.042487,11.808284,0.097222,49.568705,1.686898,1.963151


In [ ]:
offers_metrics_df = pd.merge(df_offers_grouped[['region_code', 'offer_id','discount', 'start_date', 'end_date','value_discount']],
                          results_df[['offer_id', 'subcat', 'group', 'customer_count', 'gmv_usd','discount_usd', 'quantity', 'all_customer', 'convertion_rate', 'AOV','dct_avg','margen','gmv_usd_GM','AOV_net']],
                          left_on='offer_id', right_on='offer_id')#.drop(columns = ['id_tarjeta'])

In [ ]:
def calculate_ROI(df):
    df.columns = df.iloc[7].unique()
    margen_subcat = df_margen3.loc[(df_margen3.region == df['test'].loc['region_code']) & (df_margen3.subcategoria == df['test'].loc['subcat'])].GM.unique()[0]
    
    GMV_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['test'].loc['AOV_net']# * margen_subcat
    dct_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['test'].loc['dct_avg']

    GMV_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['control'].loc['AOV_net']# * margen_subcat
    dct_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['control'].loc['dct_avg']
    
    net_GMV_t = GMV_t - dct_t
    net_GMV_c = GMV_c - dct_c
    
    diff_net_GMV = net_GMV_t - net_GMV_c
    diff_net_dct = dct_t - dct_c
    
    ROI = diff_net_GMV / diff_net_dct if diff_net_dct != 0 else 0
    
    if df['control'].loc['AOV'] != 0:
        if df['test'].loc['AOV'] == 0 or diff_net_dct < 0:
            ROI = 'NO APLICA'
        elif df['control'].loc['dct_avg']/df['control'].loc['AOV'] > df['test'].loc['dct_avg']/df['test'].loc['AOV']:
            ROI = 'NO APLICA'
    #ROI = diff_net_GMV / diff_net_dct
    return diff_net_GMV, diff_net_dct, ROI, margen_subcat

In [ ]:
for offer in offers_metrics_df.offer_id.unique():
    if offer not in (-14491634,-14491336,-14492127) and len(offers_metrics_df.loc[offers_metrics_df.offer_id == offer].T.columns) == 2:
        print(offer)
        diff_net_GMV, diff_net_dct, ROI, margen_subcat = calculate_ROI(offers_metrics_df.loc[offers_metrics_df.offer_id == offer].T)
        offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'diff_net_GMV'] = diff_net_GMV
        offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'diff_net_dct'] = diff_net_dct
        offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'ROI'] = ROI
        offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'margen_subcat'] = margen_subcat

-14487713
-14487677
-14487676
-14487675
-14487674
-14487673
-14487672
-14487671
-14487639
-14487638
-14487637
-14487636
-14487635
-14487634
-14487633
-14487632
-14487631
-14487630
-14487629
-14487628
-14487627
-14487626
-14487625
-14487624
-14487623
-14487622
-14487621
-14487620
-14487619
-14487618
-14487589
-14487588
-14487587
-14487586
-14487585
-14487584
-14487583
-14487582
-14487581
-14487580
-14487579
-14487578
-14487577
-14487576
-14487575
-14487574
-14487573
-14487572
-14487571
-14487570
-14487569
-14487568
-14487567
-14487566
-14487565
-14487564
-14487563
-14487562
-14487561
-14487560
-14487558
-14487557
-14487556
-14487555
-14487519
-14487518
-14487517
-14487516
-14487515
-14487513
-14487512
-14487511
-14487510
-14487509
-14487508
-14487507
-14487506
-14487505
-14487504
-14487503
-14487502
-14487501
-14487500
-14487499
-14487498
-14487497
-14487496
-14487495
-14487494
-14487493
-14487492
-14487491
-14487490
-14487489
-14487488
-14487487
-14487486
-14487485
-14487484
-14487483


In [ ]:
offers_metrics_df.dropna(subset=['ROI'], inplace=True)

In [ ]:
offers_metrics_df.head()

,region_code,offer_id,discount,start_date,end_date,value_discount,subcat,group,customer_count,gmv_usd,...,convertion_rate,AOV,dct_avg,margen,gmv_usd_GM,AOV_net,diff_net_GMV,diff_net_dct,ROI,margen_subcat
0,BAQ,-14487713,20230201_BAQ_comidarapida_piloto_NB_mantequill...,2023-02-01,2023-02-28,8.0,Mantequillas,control,1,5.033633,...,0.009259,5.033633,0.000000,0.174775,0.879753,0.879753,5.609646,4.026907,1.393041,0.168477
1,BAQ,-14487713,20230201_BAQ_comidarapida_piloto_NB_mantequill...,2023-02-01,2023-02-28,8.0,Mantequillas,test,3,75.504502,...,0.006865,25.168167,1.342302,0.174775,13.196293,4.398764,5.609646,4.026907,1.393041,0.168477
2,BAQ,-14487677,20230201_BAQ_fritos_piloto_NB_aceitesygrasas_W5,2023-02-01,2023-02-28,6.0,Aceites & Grasas,control,7,92.797669,...,0.175000,13.256810,0.265136,0.079473,7.374873,1.053553,15.943527,45.749043,0.3485,0.080321
3,BAQ,-14487677,20230201_BAQ_fritos_piloto_NB_aceitesygrasas_W5,2023-02-01,2023-02-28,6.0,Aceites & Grasas,test,65,1166.403713,...,0.408805,17.944673,0.817330,0.078452,91.007690,1.400118,15.943527,45.749043,0.3485,0.080321
4,BAQ,-14487676,20230201_BAQ_fritos_piloto_NB_arroz_W5,2023-02-01,2023-02-28,4.0,Arroz,control,7,346.980932,...,0.097222,49.568705,1.963151,0.042487,11.808284,1.686898,8.949785,-39.843522,NO APLICA,0.060889


In [ ]:
df_offers_grouped.head()

,region_code,offer_id,customer_segment_id_test,customer_segment_id_control,discount,start_date,end_date,value_discount,card_id,name
0,BAQ,-14487713,117440,117603,20230201_BAQ_comidarapida_piloto_NB_mantequill...,2023-02-01,2023-02-28,8.0,[253209],[Mantequilla sin Sal Alpina x 500gr]
1,BAQ,-14487712,117441,117604,20230201_BAQ_comidarapida_piloto_NB_quesos_W5,2023-02-01,2023-02-28,12.0,"[435713, 435733]",[Queso Costeño Semiduro Pasteurizado Los Campa...
2,BAQ,-14487677,117605,117526,20230201_BAQ_fritos_piloto_NB_aceitesygrasas_W5,2023-02-01,2023-02-28,6.0,"[180812, 262159]","[Aceite Naoli x 3Lt, Aceite Ricaceite x 3000ml ]"
3,BAQ,-14487676,117606,117525,20230201_BAQ_fritos_piloto_NB_arroz_W5,2023-02-01,2023-02-28,4.0,"[399349, 223861]","[Arroz Roa 25und x 500gr, Arroz Naoli x 3kg]"
4,BAQ,-14487675,117607,117527,20230201_BAQ_fritos_piloto_NB_azucaryendulzant...,2023-02-01,2023-02-28,4.0,[67822],[Azúcar en Bulto x 50kg ]


In [ ]:
df_offers_grouped.to_csv('SD_info offers.csv', index=False)
offers_metrics_df.to_csv('RESULTS_SD_ROI.csv', index=False)